In [9]:
import os
import talib as ta
import pandas as pd
import numpy as np
import datetime as dt
import alpaca_trade_api as tradeapi
import hvplot.pandas
from dotenv import load_dotenv

load_dotenv()
TICKERS = ["AAPL", "MSFT", "AMZN", "FB", "TSLA"]

In [17]:
def get_workday(date):
    """ Convert Date to its latest workday
        Parameter:
            date - Any format usagle with pd.to_datetime
        Return:
            timestamp object of the nearest workday earlier than the given date
    """
    date = pd.to_datetime(date)
    offset = max(1, (date.weekday() + 6) % 7 - 3)
    timedelta = dt.timedelta(offset)
    return pd.Timestamp((date - timedelta).date(),tz="America/New_York").isoformat()

def get_alpaca_data():
    """ Get 2 years OHLCV data from Alpaca API for the global set of tickers
        Parameter:
            None
        Return:
            Pandas Dataframe of 2 years of OHLC data for the global tickers
    """
    # Set Keys for Alpaca Trade API
    alpaca_api_key = os.getenv("ALPACA_API_KEY")
    alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")
    
    # Initiate REST API
    api = tradeapi.REST(
        alpaca_api_key,
        alpaca_secret_key,
        api_version = "v2"
    )
    
    # Set start and end date
    end_date = get_workday("today")
    two_yrs = dt.timedelta(days=(365*2))
    start_date = get_workday((pd.to_datetime("today") - two_yrs).date())
    # Set timeframe to '1D' for Alpaca API
    timeframe = "1D"
    # Get OHLCV Data
    l_portfolio = []
    for stock in TICKERS:
        l_portfolio.append(api.get_barset(
            stock,
            timeframe,
            start = start_date,
            end = end_date
        ).df)
    
    return l_portfolio

In [18]:
l_port = get_alpaca_data()
l_port

[                             AAPL                                         
                              open      high       low     close     volume
 time                                                                      
 2019-01-07 00:00:00-05:00  148.70  148.8300  145.9000  147.9100   49982480
 2019-01-08 00:00:00-05:00  149.56  151.8200  148.5200  150.6300   36592406
 2019-01-09 00:00:00-05:00  151.29  154.5300  149.6300  153.3700   40827842
 2019-01-10 00:00:00-05:00  152.50  153.9700  150.8600  153.5500   32553301
 2019-01-11 00:00:00-05:00  152.88  153.7000  151.5100  152.0534   22850904
 2019-01-14 00:00:00-05:00  150.85  151.2700  149.2200  149.7600   26038136
 2019-01-15 00:00:00-05:00  150.27  153.3900  150.0500  152.9500   25862287
 2019-01-16 00:00:00-05:00  153.08  155.8800  153.0000  154.9100   27032885
 2019-01-17 00:00:00-05:00  154.20  157.6600  153.2600  155.5300   26717099
 2019-01-18 00:00:00-05:00  157.50  157.8800  155.9806  156.5700   30450605
 2019-01-22 